## Урок 7

## Линейная регрессия. Двухвыборочный t-тест. A/B-тестирование

### Домашнее задание

В поле ответа впишите одно число. Нецелые числа следует округлить до 8 знаков после запятой.

__Задача 1__

Дана матрица объект-признак
```
X = [[ 1.22401313,  2.30868478,  3.03636353,  2.69287214],
     [-0.18757272,  1.30337355,  5.12093014,  3.46363202],
     [-0.81094525,  1.82463398,  5.79686488,  1.86159445],
     [ 0.75129018,  2.67392052,  3.65529809,  1.66746094],
     [ 0.00972362,  1.97367255,  2.50594319,  1.69755173],
     [-0.62972637,  0.77750764,  2.84124027,  4.54410559],
     [ 2.29536229,  1.81206697,  1.95026215,  1.51874636],
     [ 0.0920418 ,  2.26971361,  7.47708735,  2.61081203],
     [ 2.39252799,  3.17563985,  3.61420599,  5.10773362],
     [ 0.54983815,  2.87988651,  1.65752765,  1.59635987]]
```
и значения целевой переменной
```
y = [ 9.26193358,  9.700363  ,  8.67214805,  8.74796974,  6.18689108,
      7.53312713,  7.57643777, 12.44965478, 14.29010746,  6.68361218]
```

Подберите два признака (из четырёх) так, чтобы уровень линейной зависимости целевой переменной от значений этих признаков был максимальным. Другими словами, модель линейной регрессии на этих признаках должна давать наилучший результат. В качестве ответа запишите значение коэффициента детерминации для модели на этих признаках.

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from itertools import combinations

In [3]:
data = [
    [ 1.22401313,  2.30868478,  3.03636353,  2.69287214],
     [-0.18757272,  1.30337355,  5.12093014,  3.46363202],
     [-0.81094525,  1.82463398,  5.79686488,  1.86159445],
     [ 0.75129018,  2.67392052,  3.65529809,  1.66746094],
     [ 0.00972362,  1.97367255,  2.50594319,  1.69755173],
     [-0.62972637,  0.77750764,  2.84124027,  4.54410559],
     [ 2.29536229,  1.81206697,  1.95026215,  1.51874636],
     [ 0.0920418 ,  2.26971361,  7.47708735,  2.61081203],
     [ 2.39252799,  3.17563985,  3.61420599,  5.10773362],
     [ 0.54983815,  2.87988651,  1.65752765,  1.59635987]]

In [4]:
titles = ['1', '2', '3', '4']
index = [_ for _ in range(len(data))]

In [5]:
y = [ 9.26193358,  9.700363  ,  8.67214805,  8.74796974,  6.18689108, 
              7.53312713,  7.57643777, 12.44965478, 14.29010746,  6.68361218]

In [6]:
df = pd.DataFrame(data, index, titles)
df['y'] = y
df

,1,2,3,4,y
0,1.224013,2.308685,3.036364,2.692872,9.261934
1,-0.187573,1.303374,5.120930,3.463632,9.700363
2,-0.810945,1.824634,5.796865,1.861594,8.672148
3,0.751290,2.673921,3.655298,1.667461,8.747970
4,0.009724,1.973673,2.505943,1.697552,6.186891
5,-0.629726,0.777508,2.841240,4.544106,7.533127
6,2.295362,1.812067,1.950262,1.518746,7.576438
7,0.092042,2.269714,7.477087,2.610812,12.449655
8,2.392528,3.175640,3.614206,5.107734,14.290107
9,0.549838,2.879887,1.657528,1.596360,6.683612


In [7]:
y = df['y'].values

In [8]:
lr = LinearRegression(fit_intercept=True)

In [9]:
r2 = {}

In [10]:
def sum_of_squares(samples: np.ndarray) -> float:
    """Сумма квадратов отклонений.
    """
    
    return ((samples - samples.mean()) ** 2).sum()

In [11]:
for pair in combinations(titles, r=2):
    X = df[list(pair)].values
    lr.fit(X, y)
    y_pred = lr.predict(X)
    r2[pair] = r2_score(y, y_pred)
    #e = y - y_pred
    #R2 = 1 - sum_of_squares(e) / sum_of_squares(y)
    #r2[pair] = R2

In [12]:
r2

{('1', '2'): 0.18113594742585204,
 ('1', '3'): 0.7634246238793151,
 ('1', '4'): 0.4532966783144078,
 ('2', '3'): 0.5479482734039012,
 ('2', '4'): 0.6062055761129932,
 ('3', '4'): 0.6224419876505323}

In [13]:
max_r2 = {k:v for k, v in r2.items() if v == max(r2.values())}
max_r2

{('1', '3'): 0.7634246238793151}

In [14]:
round(max(r2.values()), 8)

0.76342462

In [15]:
r2_manual = {}

In [16]:
for pair in combinations(titles, r=2):
    
    x = df[list(pair)].values
    ones = np.ones((x.shape[0], 1))
    x = np.hstack((ones, x))
    
    XTX = x.T.dot(x)
    XTX_inv = np.linalg.inv(XTX)

    b = XTX_inv.dot(x.T).dot(y)
    z = x.dot(b)
    e = y - z
    
    R2 = 1 - sum_of_squares(e) / sum_of_squares(y)
    r2_manual[pair] = R2

In [17]:
r2_manual

{('1', '2'): 0.18113594742585204,
 ('1', '3'): 0.763424623879315,
 ('1', '4'): 0.4532966783144077,
 ('2', '3'): 0.547948273403901,
 ('2', '4'): 0.6062055761129934,
 ('3', '4'): 0.6224419876505323}

In [18]:
max_r2 = {k:v for k, v in r2_manual.items() if v == max(r2_manual.values())}
max_r2

{('1', '3'): 0.763424623879315}

In [19]:
R2 = max(max_r2.values())
round(R2, 8)

0.76342462

__Ответ:__ значение коэффициента детерминации: 0.76342462

__Задача 2__

Является ли значимым уравнение регрессии, полученное в предыдущей задаче? В качестве ответа запишите 1, если является, и 0 иначе.

Для таких целей существует т.н. __F-тест Фишера__. Формально при таком тесте проверяется нулевая гипотеза о том, что теоретический коэффициент детерминации равен $0$. Другими словами, если нулевая гипотеза верна, то между факторами и целевой переменной вообще нет никакой значимой зависимости, а отличие коэффициента детерминации от нуля обусловлено лишь случайностью процесса.

При F-тесте используется статистика:
$$F = \dfrac{R^2 / m}{(1 - R^2) / (n - m - 1)},$$
где $R^2$ — коэффициент детерминации, $n$ — число наблюдений, $m$ — число факторов. Такая статистика в предположении верности нулевой гипотезы имеет __F-распределение Фишера__ с параметрами $k_1 = m$, $k_2 = n - m - 1$.

Распределение Фишера имеет один хвост, поэтому рассматривается правосторонняя критическая область $\Omega_\alpha = \left( t_{1 - \alpha, \: k_1, k_2}, \infty \right)$, где $t_{1 - \alpha, \: k_1, k_2}$ — квантиль порядка $1 - \alpha$ для распределения Фишера с параметрами $k_1$, $k_2$.

Если статистика попадает в критическую область, то гипотеза о равенстве нулю коэффициента детерминации отвергается. Уравнение признаётся значимым.

In [20]:
from scipy import stats

In [21]:
n = df.shape[0]
m = 2 - 1  # отнимаем 1 поскольку добавляли нулевой фактор

k1 = m   
k2 = n - m - 1

k1, k2

(1, 8)

In [22]:
alpha = 0.05

t = stats.f.ppf(1 - alpha, k1, k2)
t   # квантиль порядка  1−𝛼  для распределения Фишера с параметрами  𝑘1 ,  𝑘2

5.317655071578714

Итак, критическая область:
$$\Omega_\alpha = \left( 5.32, \infty \right)$$

In [23]:
F = (R2 / k1) / ((1 - R2) / k2)
F   # статистика

25.815860852394596

In [24]:
F > t

True

Статистика попала в критическую область, следовательно нулевая гипотеза отвергается и уравнение регрессии признаётся значимым.

__Ответ__: 1

__Задача 3__

Для проведения A/B-тестирования сайта интернет-магазина были получены следующие данные: страница `A` была посещена 2509 раз, из них 77 закончились совершением покупки, страница `B` была посещена 1465 раз, 60 из них закончились совершением покупки. Является ли значимым отличие конверсии на страницах `A` и `B`? В качестве ответа запишите 1, если является, и 0 иначе.

_Подсказка_. Реализуйте двухвыборочный t-тест. В качестве выборок здесь можно взять наборы меток совершения покупки (0 или 1) каждым посетителем.

 Имеются две независимые выборки $a$, $b$, взятые. Требуется проверить гипотезу о том,  что конверсия на страницах A и B не отличается (отличие незначимо), т.е.
$$H_0: \: con_a = con_b$$

In [25]:
a = np.zeros((2509, 1))
a[0:76] = 1
b = np.zeros((1465, 1))
a[0:59] = 1

Размеры выборок равны, соответственно, $n_1$ и $n_2$. Обозначим через
$$\sigma_\Delta = \sqrt{\dfrac{\sigma_{X_1}^2}{n_1} + \dfrac{\sigma_{X_2}^2}{n_2}}$$
среднее квадратическое отклонение разности между выборочными средними выборок $X_1$ и $X_2$. Здесь $\sigma_{X_i}^2$ — несмещённая оценка дисперсии по выборке $X_i$.

In [26]:
na = a.size   # размер выборки a
nb = b.size   # размер выборк b

sa = a.std(ddof=1)   #несмещенное среднеквадратичное отклонение по выборке a
sb = b.std(ddof=1)   #несмещенное среднеквадратичное отклонение по выборке b

In [27]:
s_delta = np.sqrt(sa ** 2 / na + sb ** 2 / nb)
s_delta   # среднее квадратическое отклонение разности между выборочными средними выборок

0.00342226332551712

Для проверки данной гипотезы используется следующая __статистика__:
$$t = \dfrac{\overline{X_1} - \overline{X_2}}{\sigma_\Delta}$$

In [28]:
t = (a.mean() - b.mean()) / s_delta

t   # значение статистики

8.851146066081345

В предположении нулевой гипотезы данная статистика имеет распределение Стьюдента. Число степеней свободы распределения определяется из следующего равенства:
$$df = \dfrac{\left( \dfrac{\sigma_{X_1}^2}{n_1} + \dfrac{\sigma_{X_2}^2}{n_2} \right)^2}{\dfrac{\left( \sigma_{X_1}^2 / n_1 \right)^2}{n_1 - 1} + \dfrac{\left( \sigma_{X_2}^2 / n_2 \right)^2}{n_2 - 1}}$$

Зафиксируем уровень значимости $\alpha = 0.05$. Теперь нужно посчитать квантили распределения Стьюдента. Посчитаем число степеней свободы этого распределения:

In [29]:
df = (sa ** 2 / na + sb ** 2 / nb) ** 2 / \
    ((sa ** 2 / na) ** 2 / (na - 1) + (sb ** 2 / nb) ** 2 / (nb - 1))

df

2508.0

In [30]:
alpha = 0.05

t1 = stats.t.ppf(alpha / 2, df=df)
t2 = stats.t.ppf(1 - alpha / 2, df=df)

t1, t2

(-1.9609103150962355, 1.960910315096235)

Итак, критическая область имеет вид:
$$\Omega_\alpha = (-\infty, -1.96) \cup (1.96, \infty)$$

Значение статистики равно $8.85$ и попало в критическую область. Гипотеза о равенстве конверсий отвергается. Значит, утверждение о том, что отличие конверсии на страницах A и B незначимо, ложно.

In [31]:
stats.ttest_ind(a, b, equal_var=False)

Ttest_indResult(statistic=array([8.85114607]), pvalue=array([1.60169206e-18]))

__Ответ__: 0